In [1]:
!pip install numpy
!pip install pandas
!pip install keras

In [2]:
import numpy as np
import pandas as pd

In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [7]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

In [12]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [13]:
from keras.models import Sequential
from keras.layers import Dense

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [15]:
from sklearn.model_selection import train_test_split

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [16]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [17]:
# build the model
model = regression_model()

In [18]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=2)



Epoch 1/50


2024-11-03 23:43:30.492278: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-11-03 23:43:30.497475: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394315000 Hz
2024-11-03 23:43:30.498039: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55fdbe0674a0 executing computations on platform Host. Devices:
2024-11-03 23:43:30.498081: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-11-03 23:43:30.567964: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 1613.4012
Epoch 2/50
 - 0s - loss: 1595.6621
Epoch 3/50
 - 0s - loss: 1578.7345
Epoch 4/50
 - 0s - loss: 1562.2013
Epoch 5/50
 - 0s - loss: 1545.5406
Epoch 6/50
 - 0s - loss: 1529.0728
Epoch 7/50
 - 0s - loss: 1512.4227
Epoch 8/50
 - 0s - loss: 1495.3757
Epoch 9/50
 - 0s - loss: 1477.8605
Epoch 10/50
 - 0s - loss: 1459.7680
Epoch 11/50
 - 0s - loss: 1441.2097
Epoch 12/50
 - 0s - loss: 1422.1732
Epoch 13/50
 - 0s - loss: 1402.2433
Epoch 14/50
 - 0s - loss: 1381.9463
Epoch 15/50
 - 0s - loss: 1360.8418
Epoch 16/50
 - 0s - loss: 1338.9132
Epoch 17/50
 - 0s - loss: 1316.3768
Epoch 18/50
 - 0s - loss: 1293.7216
Epoch 19/50
 - 0s - loss: 1270.0744
Epoch 20/50
 - 0s - loss: 1245.8709
Epoch 21/50
 - 0s - loss: 1220.9206
Epoch 22/50
 - 0s - loss: 1196.0692
Epoch 23/50
 - 0s - loss: 1170.6843
Epoch 24/50
 - 0s - loss: 1144.9208
Epoch 25/50
 - 0s - loss: 1119.0260
Epoch 26/50
 - 0s - loss: 1092.5686
Epoch 27/50
 - 0s - loss: 1066.5214
Epoch 28/50
 - 0s - loss: 1040.1318
Epoch 29/50


In [19]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 88us/step


499.0038136108794

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

499.0038211114724 0.0


In [22]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 175.90782348237764
MSE 2: 146.3684756578365
MSE 3: 92.73696706827404
MSE 4: 72.33290901307535
MSE 5: 58.85228847762913
MSE 6: 56.014235203320155
MSE 7: 53.38111818183973
MSE 8: 37.06135368964433
MSE 9: 39.177960985301
MSE 10: 38.73950247162754
MSE 11: 36.869727310625095
MSE 12: 35.59011322317772
MSE 13: 41.37532303325567
MSE 14: 42.43351188832502
MSE 15: 34.53048645562724
MSE 16: 30.70479507816648
MSE 17: 33.45650232185438
MSE 18: 32.2983673027804
MSE 19: 32.785148602087524
MSE 20: 30.71644936718987
MSE 21: 32.03827859822986
MSE 22: 31.12783008563094
MSE 23: 28.319140733638626
MSE 24: 28.12198424725085
MSE 25: 32.953393158403415
MSE 26: 34.678708443749684
MSE 27: 29.379525835845847
MSE 28: 29.56321424342282
MSE 29: 34.53943461199023
MSE 30: 32.143897639894945
MSE 31: 29.468837849144798
MSE 32: 26.090748345581844
MSE 33: 27.794362855189053
MSE 34: 30.191544900048513
MSE 35: 30.06048643241808
MSE 36: 34.91390209753536
MSE 37: 27.19408184347801
MSE 38: 33.905057444155794
MSE 39: 32